# 1st part - The gps coordinates

2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

3. To create the above dataframe:

   * The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
   * Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
   * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
   * If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
   * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
   * In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4. Submit a link to your Notebook on your Github repository. (10 marks)

## 1 Import of the libraries

In [1]:
# install the package for the 'soup'
# see https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser for a comparison
#!pip install lxml
#!pip install html5lib
# import the libraries (soup + usual) 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

## 2 Reading the html of the wiki page

In [2]:
source=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup=BeautifulSoup(source, 'lxml')

Extract the table information and store it as a dataframe

In [3]:
data=[]
header=[]
table= soup.find('table',attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')
rows = table_body.find_all('tr')
#print(table_body.prettify())

for row in rows:
    cols = row.find_all('td')
    head = row.find_all('th')
    cols = [ele.text.strip() for ele in cols]
    heads= [ele.text.strip() for ele in head]
    data.append([ele for ele in cols if ele]) # Get rid of empty values
    header.append([ele for ele in heads if ele])

del(data[0]) # supprimer la ligne de départ
# get the header list (without the empty cells)
header= list(filter(None,header))
# convertir les 2 listes en dataframe
df = pd.DataFrame(data, columns=header[0])

## 3 Fullfilling the several requests:
* if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [4]:
deleted='Not assigned'
for row in df.itertuples():    
    if row.Neighbourhood == deleted:
        df.Neighbourhood.iloc[row.Index] = df.Borough.iloc[row.Index]

* Ignore cells with a borough that is Not assigned.

In [5]:
df.drop(df[df['Borough']=='Not assigned'].index, inplace=True)

* Merging similar Postcode 'These two rows will be combined into one row with the neighborhoods separated with a comma'

In [6]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()

Here is an overview of the table:

In [7]:
df.rename(columns={'Postcode':'PostalCode'}, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [8]:
print('There is',df.shape[0], 'rows and',df.shape[1],'columns in the dataframe.')

There is 103 rows and 3 columns in the dataframe.
